___

<a href='https://mainacademy.ua/'> <img src='https://mainacademy.ua/wp-content/uploads/2019/03/logo-main.png' alt = 'Хмм, щось з інтернетом'/></a>
___

# Module 4: Work with data professionally!

## Lab work 4



#### Мета: 

* навчитися обробляти дані
* "парсити" сайти

### Завдання 1:

#### Ознайомитися із API Національного банку 
https://old.bank.gov.ua/control/uk/publish/article?art_id=38441973&cat_id=38459171#exchange



1. Підключитися до API.


In [ ]:
import requests
import json
response_API = requests.get('https://bank.gov.ua/NBUStatService/v1/statdirectory/exchangenew?json?valcode=USD&date=20230103')
print(response_API.status_code)

2. Отримати курс долара за останній рік.

In [ ]:
import requests
import json
import pandas as pd
from datetime import datetime, timedelta
import statistics as stats

url = "http://bank.gov.ua/NBUStatService/v1/statdirectory/exchangenew?json"

response_API = requests.get(url)
#print(response_API.status_code)

data = response_API.text #helps us pull the data from the mentioned API.

data_json=json.loads(data) #convert and decode the data into proper JSON format

how_many_days = int(input('Enter number of days' ))
date = datetime.today()

rate=[]

for j in range(how_many_days):
    date = date - timedelta(days = 1)
    date_for_req = {'date' : date.strftime("%Y%m%d")}
    data_json = requests.get(url, params = date_for_req).json()
    for i in data_json:
         for k in i.values():
              if k=='USD':
                  rate+=[i['rate']]
                  print('The rate of US Dollar ', i['rate'], 'on ', i['exchangedate'])

3. Вивести середнє значення та відхилення курсу за кожний місяць.

In [8]:
import requests
import json
import pandas as pd
from datetime import datetime, timedelta
import statistics as stats

url = "http://bank.gov.ua/NBUStatService/v1/statdirectory/exchangenew?json"

response_API = requests.get(url)
#print(response_API.status_code)

data = response_API.text #helps us pull the data from the mentioned API.

data_json=json.loads(data) #convert and decode the data into proper JSON format

how_many_days = int(input('Enter number of days' ))
date = datetime.today()

rate=[]

for j in range(how_many_days):
    date = date - timedelta(days = 1)
    date_for_req = {'date' : date.strftime("%Y%m%d")}
    data_json = requests.get(url, params = date_for_req).json()
    for i in data_json:
         for k in i.values():
              if k=='USD':
                  rate+=[i['rate']]
                  #print('The rate of US Dollar ', i['rate'], 'on ', i['exchangedate'])

print('The avarage rate over the past',how_many_days, 'days is %.2f'%stats.mean(rate), 'and the standard deviation is %.2f'%stats.variance(rate))    


Enter number of days7
The avarage rate over the past 7 days is 36.5686 and the standard deviation is  0.0


4. Дану інформацію записати у файл за допомогою pickle.

In [ ]:
import pickle
file=open('US Rates.py','wb')
pickle.dump(rate,file)
file.close()

### Завдання 2:

Потрібно проаналізувати всі товари на сайті: 
https://smallpacking.agrosem.ua/shop/

Створіть базу даних, що містить таблицю із такими стовбцями: 
    - артикул товару
    - найменування
    - ціна 
    - вага

In [ ]:
import requests
import json
import pandas as pd
from datetime import datetime, timedelta
import statistics as stats

from bs4 import BeautifulSoup as bs

url='https://smallpacking.agrosem.ua/products'
price=[]
product=[]
for i in [1,2,3,4,5]:
    i=str(i)
    url='https://smallpacking.agrosem.ua/products/?page='+i
    print(url)
    response= requests.get(url)
    #print(response.status_code)
    soup = bs(response.content,'html.parser')
    rev_price=soup.findAll("div",attrs={"class","order"})
    rev_prod=soup.findAll("div",attrs={"class","info"})
    for j in range(len(rev_price)):#extracts the price
         price.append(rev_price[j].find('p').text)
    for j in range(len(rev_prod)-1):#extracts the product name
      product.append(rev_prod[j].find('a').text)    


int=[]
for j in range(len(product)):
    int+=[s for s in product[j].split() if s.isdigit()]



#create a table in python
data={'Product': product, 'Weight (kg)' : int, 'Price' : price}
df=pd.DataFrame(data=data)

df


### Завдання 3: 

Ознайомтеся із роботою SQLite та відповідним модулем у Python.
Завантажте базу даних для виконання лабораторної роботи.
Підключіться до завантаженої бази SQLite.

1. Виведіть інформацію про дану базу.

In [110]:
import sqlite3
import pandas as pd
conn=sqlite3.connect("tysql.sqlite")
conn

2. Виведіть перелік всіх таблиць.

In [111]:
from pandas import read_sql_query, read_sql_table
tables = list(read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)['name'])
out = {tbl : read_sql_query(f"SELECT * from {tbl}", conn) for tbl in tables}
print(out)

{'Customers':       cust_id      cust_name          cust_address cust_city cust_state  \
0  1000000001   Village Toys        200 Maple Lane   Detroit         MI   
1  1000000002     Kids Place  333 South Lake Drive  Columbus         OH   
2  1000000003        Fun4All         1 Sunny Place    Muncie         IN   
3  1000000004        Fun4All   829 Riverside Drive   Phoenix         AZ   
4  1000000005  The Toy Store      4545 53rd Street   Chicago         IL   

  cust_zip cust_country        cust_contact             cust_email  
0    44444          USA          John Smith  sales@villagetoys.com  
1    43333          USA      Michelle Green                   None  
2    42222          USA           Jim Jones     jjones@fun4all.com  
3    88888          USA  Denise L. Stephens  dstephens@fun4all.com  
4    54545          USA          Kim Howard                   None  , 'OrderItems':     order_num  order_item prod_id  quantity  item_price
0       20005           1    BR01       100       

3. Список всіх cust_id з таблиці Customers table.

In [8]:
report_customer_table=out["Customers"]
report_customer_table.loc[:,"cust_id"]

0    1000000001
1    1000000002
2    1000000003
3    1000000004
4    1000000005
Name: cust_id, dtype: object

4. Всю таблицю Customers table

In [9]:
print(out["Customers"])

      cust_id      cust_name          cust_address cust_city cust_state  \
0  1000000001   Village Toys        200 Maple Lane   Detroit         MI   
1  1000000002     Kids Place  333 South Lake Drive  Columbus         OH   
2  1000000003        Fun4All         1 Sunny Place    Muncie         IN   
3  1000000004        Fun4All   829 Riverside Drive   Phoenix         AZ   
4  1000000005  The Toy Store      4545 53rd Street   Chicago         IL   

  cust_zip cust_country        cust_contact             cust_email  
0    44444          USA          John Smith  sales@villagetoys.com  
1    43333          USA      Michelle Green                   None  
2    42222          USA           Jim Jones     jjones@fun4all.com  
3    88888          USA  Denise L. Stephens  dstephens@fun4all.com  
4    54545          USA          Kim Howard                   None  


5. Список клієнтів (cust_names) відсортованих від Z до A

In [10]:
sorted(report_customer_table.loc[:,"cust_name"],reverse=True)

['Village Toys', 'The Toy Store', 'Kids Place', 'Fun4All', 'Fun4All']

6. Таблицю клієнтів та замовлень (cust_id і order_num) . Відсортуйте по клієнту і потім по даті замовлення

In [16]:
l=list(report_customer_table.loc[:,"cust_id"])
ll=list(report_order_table.loc[:,"order_num"])
df=pd.DataFrame({"Customers ID":l,"Order Number":ll})
print("Customers and their Orders \n",df)


Customers and their Orders 
   Customers ID  Order Number
0   1000000001         20005
1   1000000002         20006
2   1000000003         20007
3   1000000004         20008
4   1000000005         20009


7. Таблицю (на основі Items) з кількість та вартістю товару. Відсортуйте в порядку спадання по кількості та вартості

In [27]:
record_orderitems_table=out["OrderItems"]
l=list(record_orderitems_table.loc[:,"quantity"])
ll=list(record_orderitems_table.loc[:,"item_price"])
#sorted(report_customer_table.loc[:,"cust_name"],reverse=True)
df=pd.DataFrame({"Quantity of Product":l,"Price":ll})

dfq=df.sort_values(by=['Quantity of Product'])
print('Sort by quantity\n',dfq)

dfp=df.sort_values(by=['Price'])
print('\nSort by price ')
dfp

Sort by quantity
     Quantity of Product  Price
11                    5  11.99
10                    5   4.99
14                   10   3.49
3                    10   8.99
4                    10  11.99
13                   10   3.49
12                   10   3.49
2                    20   5.99
5                    50  11.49
9                    50   4.49
0                   100   5.49
8                   100   2.99
7                   100   2.99
6                   100   2.99
1                   100  10.99
15                  250   2.49
16                  250   2.49
17                  250   2.49

Sort by price 


,Quantity of Product,Price
17,250,2.49
15,250,2.49
16,250,2.49
7,100,2.99
6,100,2.99
8,100,2.99
12,10,3.49
13,10,3.49
14,10,3.49
9,50,4.49


8. Товар (з таблиці Products), ціна якого становить 9.49

In [28]:
record_product_table=out["Products"]
product_name = record_product_table.loc[record_product_table["prod_price"]==9.49]["prod_name"]
product_name

7     King doll
8    Queen doll
Name: prod_name, dtype: object

9. Виведіть назву товару та ціну, яка лежить в діапазоні від  3 до 6. Відсортуйте результат в по ціні в порядку зростання

In [29]:
value9 = record_product_table.loc[record_product_table["prod_price"].between(3,6)]["prod_name"]
value9

0      8 inch teddy bear
3      Fish bean bag toy
4      Bird bean bag toy
5    Rabbit bean bag toy
6            Raggedy Ann
Name: prod_name, dtype: object

10. Кількість товару, що було продано

In [30]:
#l=list(record_orderitems_table.loc[:,"quantity"])
print("The total amound of sold items: ", sum(l))

The total amound of sold items:  1430


11. Кількість найменувань товару, ціна якого більша за 4

In [95]:
len(record_product_table.loc[record_product_table["prod_price"]>4]["prod_name"])


6

12. Розробіть алгоритм для виведення 3 найдорожчих товарів в базі

In [28]:
record_product_table.sort_values(["prod_price"],ascending=False)[:3]
                                                                

,prod_id,vend_id,prod_name,prod_price,prod_desc
2,BR03,BRS01,18 inch teddy bear,11.99,"18 inch teddy bear, comes with cap and jacket"
7,RYL01,FNG01,King doll,9.49,12 inch king doll with royal garments and crown
8,RYL02,FNG01,Queen doll,9.49,12 inch queen doll with royal garments and crown


13. Підрахуйте кількість замовлень для кожного клієнта використовуючи підзапити

In [87]:
k=list(record_orderitems_table.loc[:,"order_num"])
kk=list(record_orderitems_table.loc[:,"quantity"])
dd=pd.DataFrame({"ord":k, "total":kk})
t=dd.groupby(['ord']).sum()
t=list(t.loc[:,'total'])

df=pd.DataFrame({'Customer ID': l,'Number of purchases': t})
df

,Customer ID,Number of purchases
0,1000000001,200
1,1000000003,40
2,1000000004,400
3,1000000005,40
4,1000000001,750


14. Виведіть список клієнтів та їх замовлення (використайте JOIN)

In [93]:
k=list(record_orderitems_table.loc[:,"order_num"])
kk=list(record_orderitems_table.loc[:,"prod_id"])
dd=pd.DataFrame({"ord":k, "total":kk})

dfb = dd.groupby(['ord']).agg(','.join)

tt=list(dfb.loc[:,'total'])
df=pd.DataFrame({'Customer ID': l,'Purchased Items': tt})
df

,Customer ID,Purchased Items
0,1000000001,"BR01,BR03"
1,1000000003,"BR01,BR02,BR03"
2,1000000004,"BR03,BNBG01,BNBG02,BNBG03,RGAN01"
3,1000000005,"RGAN01,BR03,BNBG01,BNBG02,BNBG03"
4,1000000001,"BNBG01,BNBG02,BNBG03"


15. Зробіть два запити: список товарів, ціна яких менша 5 та список товарів, ціна який більша рівна 5. Використайте UNION для об'єднання цих запитів

In [109]:
a1=set(record_product_table.loc[record_product_table["prod_price"]<5]["prod_name"])
a2=set(record_product_table.loc[record_product_table["prod_price"]>=5]["prod_name"])
#print("Price is less than 5 and price is greater than 5 ", a1.union(a2))
print(a1.union(a2))


{'8 inch teddy bear', 'Fish bean bag toy', 'Rabbit bean bag toy', '18 inch teddy bear', 'Queen doll', 'Raggedy Ann', '12 inch teddy bear', 'King doll', 'Bird bean bag toy'}


### Завдання 4: 

Зареєструйтеся на сайті  http://www.sql-ex.ru/

Виконайте завдання із блоку SELECT; з 1 по 30 завдання

Код запитів вставляйте сюди


In [ ]:
#do not attemp